In [ ]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


以 surprise model 做 模型預測 與 用 SVD 奇異值分解 當比較

In [ ]:
import os
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import PredefinedKFold

In [ ]:
!wget -nc http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -n ml-100k.zip

File ‘ml-100k.zip’ already there; not retrieving.

Archive:  ml-100k.zip


In [ ]:
files_dir = ('ml-100k/')
reader = Reader('ml-100k')


train_file = files_dir + 'u%d.base'
test_file = files_dir + 'u%d.test'
folds_files = [(train_file % i, test_file % i) for i in (1, 2, 3, 4, 5)]

#載入數據集
data = Dataset.load_from_folds(folds_files, reader=reader)
pkf = PredefinedKFold()

algo = SVD()

for trainset, testset in pkf.split(data):
    # 訓練與測試
    algo.fit(trainset)
    predictions = algo.test(testset)

    # 計算均方根誤差
    accuracy.rmse(predictions, verbose=True)


RMSE: 0.9543
RMSE: 0.9409
RMSE: 0.9331
RMSE: 0.9326
RMSE: 0.9348


In [ ]:
import pandas as pd
#Load u.data 
rating_header = ["user_id","item_id","rating","timestamp"]
rating = pd.read_csv("ml-100k/u.data",sep="\t",header = None , names=rating_header)

uid = str(880)
iid = str(476)
#display(rating)
m1 = rating['item_id']==476
m2 = rating['user_id']==880
display(rating[m1&m2])
pred = algo.predict(uid,iid,r_ui=3,verbose=True)


,user_id,item_id,rating,timestamp
99995,880,476,3,880175444


user: 880        item: 476        r_ui = 3.00   est = 3.53   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


In [ ]:
from surprise import Reader, Dataset, SVD, NormalPredictor, BaselineOnly, KNNBasic, NMF
from surprise.model_selection import cross_validate, KFold ,GridSearchCV , RandomizedSearchCV

reader = Reader(rating_scale=(1, 5))
sup_data = Dataset.load_from_df(rating[['user_id', 'item_id', 'rating']], reader)

algo = NormalPredictor()
cross_validate(algo, sup_data,measures=['RMSE', 'MAE'], cv=5, verbose=True)

for trainset, testset in pkf.split(data):
    # 訓練與測試
    algo.fit(trainset)
    predictions = algo.test(testset)

    # 計算均方根誤差
    accuracy.rmse(predictions, verbose=True)
pred = algo.predict(uid,iid,r_ui=3,verbose=True)


Evaluating RMSE, MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5179  1.5219  1.5185  1.5266  1.5184  1.5207  0.0033  
MAE (testset)     1.2173  1.2215  1.2163  1.2232  1.2145  1.2186  0.0033  
Fit time          0.16    0.62    0.23    0.25    0.24    0.30    0.16    
Test time         0.32    0.54    0.36    0.18    0.18    0.31    0.14    
RMSE: 1.5257
RMSE: 1.5275
RMSE: 1.5072
RMSE: 1.5070
RMSE: 1.5208
user: 880        item: 476        r_ui = 3.00   est = 3.74   {'was_impossible': False}


In [ ]:
def print_predict():
  uid = str(880)
  iid = str(476)
  #display(rating)
  m1 = rating['item_id']==476
  m2 = rating['user_id']==880
  #display(rating[m1&m2])
  pred = algo.predict(uid,iid,r_ui=3,verbose=True)

In [ ]:
algo = SVD()
cross_validate(algo, sup_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9405  0.9379  0.9383  0.9350  0.9263  0.9356  0.0050  
MAE (testset)     0.7395  0.7384  0.7389  0.7380  0.7314  0.7373  0.0030  
Fit time          1.98    1.97    2.12    2.50    1.97    2.11    0.21    
Test time         0.19    0.32    0.43    0.17    0.18    0.26    0.10    


{'test_rmse': array([0.94046995, 0.93787044, 0.93831977, 0.93495705, 0.92626739]),
 'test_mae': array([0.73948114, 0.73844513, 0.73892642, 0.73803726, 0.73141751]),
 'fit_time': (1.9805586338043213,
  1.9659438133239746,
  2.1226582527160645,
  2.503816604614258,
  1.9667842388153076),
 'test_time': (0.1868596076965332,
  0.3209724426269531,
  0.43195152282714844,
  0.1681206226348877,
  0.17647385597229004)}

In [ ]:
print_predict()

user: 880        item: 476        r_ui = 3.00   est = 3.53   {'was_impossible': False}
